In [60]:
# 영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.
# LLM은 항상 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.
# 예제를 제공하려면 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하세요.
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "movie": "Godfather",
        "answer": """
        Here is what I know:
        Director: Francis Ford Coppola
        Actors: Marlon Brando (Vito Corleone), Al Pacino (Michael Corleone), James Caan (Sonny Corleone), Robert Duvall (Tom Hagen), Diane Keaton (Kay Adams)
        budget: Approximately $6 million
        Genre: Crime, Drama
        Synopsis: "\"The Godfather\" is a 1972 American crime film directed by Francis Ford Coppola, based on Mario Puzo's novel of the same name. The story, set primarily between 1945 and 1955, chronicles the Corleone family under the patriarch Vito Corleone, focusing upon the transformation of his youngest son, Michael Corleone, from reluctant family outsider to ruthless mafia boss. The movie explores themes of family, power, corruption, and the American dream."
        """,
    },
    {
        "movie": "The Butterfly Effect",
        "answer": """
        I know this:
        Director: Eric Bress, J. Mackye Gruber
        Actors: Ashton Kutcher (Evan Treborn), Amy Smart (Kayleigh Miller), Eric Stoltz (George Miller), William Lee Scott (Tommy Miller), Elden Henson (Lenny)
        budget: Approximately $13 million
        Genre: Thriller, Sci-Fi
        Synopsis: "\"The Butterfly Effect\" is a 2004 American science fiction thriller film directed by Eric Bress and J. Mackye Gruber. The film stars Ashton Kutcher as Evan Treborn, a young man who discovers he can travel back in time and inhabit his younger self, allowing him to change the past. However, he soon finds that even the smallest changes can have drastic and unforeseen consequences on the present."
        """,
    },
    {
        "movie": "8 mile",
        "answer": """
        I know this:
        Director: Curtis Hanson
        Actors: Eminem (Jimmy "B-Rabbit" Smith Jr.), Kim Basinger (Stephanie Smith), Brittany Murphy (Alex Latorski), Mekhi Phifer (Future)
        budget: Approximately $41 million
        Genre: Drama, Musical
        Synopsis: "\"8 Mile\" is a 2002 American drama film directed by Curtis Hanson and starring Eminem, Mehki Phifer, Brittany Murphy, and Kim Basinger. The film follows the story of Jimmy "B-Rabbit" Smith Jr., a young white rapper living in Detroit in 1995. The title is derived from 8 Mile Road, the highway between the predominantly black city of Detroit and the primarily white suburbs, a metaphor for the social divisions that the protagonist has to overcome. The movie shows his attempts to launch a career in hip hop, a genre dominated by black artists, and the struggles he faces."
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Do you know about the movie The {movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert on all films regardless of genre, you give short answers.",
        ),
        example_prompt,
        ("human", "What do you know about {movie}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie": "The Bourne Ultimatum"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "movie": "The Bourne Ultimatum"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "movie": "The Bourne Ultimatum"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "SystemMessage"
        ],
        "kwargs": {
          "content": "You are an expert on all films regardless of genre, you give short answers.",
          "additional_kwargs": {}
        }
      },
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          

AIMessageChunk(content='\n        I know this:\n        Director: Paul Greengrass\n        Actors: Matt Damon (Jason Bourne), Julia Stiles (Nicky Parsons), David Strathairn (Noah Vosen), Joan Allen (Pamela Landy), Albert Finney (Dr. Albert Hirsch)\n        budget: Approximately $110 million\n        Genre: Action, Thriller\n        Synopsis: ""The Bourne Ultimatum" is a 2007 American action thriller film directed by Paul Greengrass and starring Matt Damon as Jason Bourne. It is the third installment in the Bourne series, following Bourne as he continues to uncover his past and the government conspiracy surrounding the Treadstone project. The film is known for its intense action sequences, fast-paced editing, and intricate plot that keeps audiences on the edge of their seats."\n        ')